<a href="https://colab.research.google.com/github/trinade96/Fortiate_Project/blob/master/Fortiate_DST_3_and_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Trina De

##### **Packages Used**

In [0]:
import numpy as np 
import pandas as pd
from datetime import date
import time

##### **Loading the data**

In [2]:
from google.colab import drive
drive.mount('/gdrive',force_remount=True)
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


##### **Reading the Data**

In [3]:
missing_values = ["n/a", "na", "--", ' ', "NAN","nan","NA","NaN"]
data = pd.read_excel("/gdrive/My Drive/DATA FOR ALL TEAMS-25MB.xlsx",na_values=missing_values)
data.head()

,Card-Wallet-Token ID,PROCESSING_CODE,TRANSACTION_AMOUNT,SETTLEMENT_AMOUNT,BILLING_AMOUNT,Transaction Date and Time,CONVERSION_RATE,Expiry Date,CONVERSION_RATE_DATE,Merchant Category Code,ACQUIRING_COUNTRY_CODE,POS entry Mode,ACQUIRER_INSTITUTION_CODE,Authorization Code,Approval Decision,Card Service Code,TID,MID,MERCHANT NAME AND ADDRESS,Private Data 48,Transaction Category Code,DE48 Data Length,TRANSACTION_CURRENCY,SETTLEMENT_CURRENCY,BILLING_CURRENCY,POS_DATA,Interface ID,Terminal Type,Token Data-C1,Additional Payment Info,Ecommerce Data,ECI Indicator,Remaining Balance,Private Data - Additional Info,Token Data,TARGET
0,DE2,DE3,DE4,DE5,DE6,DE7,DE9,DE14,DE16,DE18,DE19,DE22,DE32,DE38,DE39,DE40,DE41,DE42,DE43,DE48,Private Data 48-1,Private Data 48-2,DE49,DE50,DE51,DE61,DE4801,DE4801,DE4803,DE4804,DE4805,DE4806,DE4807,DE4808,DE4809,DE4810
1,6086941819072006,174000,7000,7000,7000,22-MAR-18 09.10.37,0,01-JUL-23 00.00.00,NaN,6011,356,???,544921,533038,000,201,GYACBB04,1005044921,LXFC BTNK LTD.KHURDT OR IN,NaN,NaN,NaN,356,356,356,210201210141,IN1,ATM,0917,NaN,NaN,NaN,35000,NaN,NaN,NaN
2,6086941026163002,174000,2000,2000,2000,22-MAR-18 12.28.15,0,01-DEC-22 00.00.00,NaN,6011,356,???,544921,944896,000,201,JSEWLK06,1005044921,LXFC BTNK LTD.LUCKNOWUP IN,NaN,NaN,NaN,356,356,356,210201210141,IN1,ATM,0796,NaN,NaN,NaN,44000,NaN,NaN,NaN
3,6086941026163002,174000,1000,1000,1000,22-MAR-18 14.51.45,0,01-DEC-22 00.00.00,NaN,6011,356,???,544921,153505,000,201,JSEWLK06,1005044921,LXFC BTNK LTD.LUCKNOWUP IN,NaN,NaN,NaN,356,356,356,210201210141,IN1,ATM,0796,NaN,NaN,NaN,42000,NaN,NaN,NaN
4,6086941055683003,174000,4500,28.59,2034.06,22-MAR-18 15.50.10,56874000,01-JUL-23 00.00.00,NaN,6011,144,???,544921,757011,000,201,A9021311,1144000031,STMPTTH BTNK TTMCTSHLK,NaN,NaN,NaN,144,840,356,210201210141,IN1,ATM,0797,NaN,NaN,NaN,14791.4,NaN,NaN,NaN


In [4]:
data2 = data.iloc[1:,:]
data2.head()

,Card-Wallet-Token ID,PROCESSING_CODE,TRANSACTION_AMOUNT,SETTLEMENT_AMOUNT,BILLING_AMOUNT,Transaction Date and Time,CONVERSION_RATE,Expiry Date,CONVERSION_RATE_DATE,Merchant Category Code,ACQUIRING_COUNTRY_CODE,POS entry Mode,ACQUIRER_INSTITUTION_CODE,Authorization Code,Approval Decision,Card Service Code,TID,MID,MERCHANT NAME AND ADDRESS,Private Data 48,Transaction Category Code,DE48 Data Length,TRANSACTION_CURRENCY,SETTLEMENT_CURRENCY,BILLING_CURRENCY,POS_DATA,Interface ID,Terminal Type,Token Data-C1,Additional Payment Info,Ecommerce Data,ECI Indicator,Remaining Balance,Private Data - Additional Info,Token Data,TARGET
1,6086941819072006,174000,7000,7000,7000,22-MAR-18 09.10.37,0,01-JUL-23 00.00.00,NaN,6011,356,???,544921,533038,000,201,GYACBB04,1005044921,LXFC BTNK LTD.KHURDT OR IN,NaN,NaN,NaN,356,356,356,210201210141,IN1,ATM,0917,NaN,NaN,NaN,35000,NaN,NaN,NaN
2,6086941026163002,174000,2000,2000,2000,22-MAR-18 12.28.15,0,01-DEC-22 00.00.00,NaN,6011,356,???,544921,944896,000,201,JSEWLK06,1005044921,LXFC BTNK LTD.LUCKNOWUP IN,NaN,NaN,NaN,356,356,356,210201210141,IN1,ATM,0796,NaN,NaN,NaN,44000,NaN,NaN,NaN
3,6086941026163002,174000,1000,1000,1000,22-MAR-18 14.51.45,0,01-DEC-22 00.00.00,NaN,6011,356,???,544921,153505,000,201,JSEWLK06,1005044921,LXFC BTNK LTD.LUCKNOWUP IN,NaN,NaN,NaN,356,356,356,210201210141,IN1,ATM,0796,NaN,NaN,NaN,42000,NaN,NaN,NaN
4,6086941055683003,174000,4500,28.59,2034.06,22-MAR-18 15.50.10,56874000,01-JUL-23 00.00.00,NaN,6011,144,???,544921,757011,000,201,A9021311,1144000031,STMPTTH BTNK TTMCTSHLK,NaN,NaN,NaN,144,840,356,210201210141,IN1,ATM,0797,NaN,NaN,NaN,14791.4,NaN,NaN,NaN
5,6086941020177008,004000,499,7.14,499,22-MAR-18 02.35.44,0,01-DEC-22 00.00.00,NaN,5967,840,???,370100,954345,000,201,FN90G440,9590108460,GOOGLE SERDICES \\MOUNTTIN DIEW\94043 \,NaN,NaN,NaN,356,840,356,200111100040,IN1,POS,0797,NaN,NaN,NaN,177495,NaN,NaN,NaN


##### **Changing non NaNs in 'TARGET' column to 'Not Fraud'**

In [5]:
data2['TARGET'] = [x if x == 'Fraud' else 'Not Fraud' for x in list(data2['TARGET'])]
set(data2['TARGET'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


{'Fraud', 'Not Fraud'}

In [6]:
data2.head()

,Card-Wallet-Token ID,PROCESSING_CODE,TRANSACTION_AMOUNT,SETTLEMENT_AMOUNT,BILLING_AMOUNT,Transaction Date and Time,CONVERSION_RATE,Expiry Date,CONVERSION_RATE_DATE,Merchant Category Code,ACQUIRING_COUNTRY_CODE,POS entry Mode,ACQUIRER_INSTITUTION_CODE,Authorization Code,Approval Decision,Card Service Code,TID,MID,MERCHANT NAME AND ADDRESS,Private Data 48,Transaction Category Code,DE48 Data Length,TRANSACTION_CURRENCY,SETTLEMENT_CURRENCY,BILLING_CURRENCY,POS_DATA,Interface ID,Terminal Type,Token Data-C1,Additional Payment Info,Ecommerce Data,ECI Indicator,Remaining Balance,Private Data - Additional Info,Token Data,TARGET
1,6086941819072006,174000,7000,7000,7000,22-MAR-18 09.10.37,0,01-JUL-23 00.00.00,NaN,6011,356,???,544921,533038,000,201,GYACBB04,1005044921,LXFC BTNK LTD.KHURDT OR IN,NaN,NaN,NaN,356,356,356,210201210141,IN1,ATM,0917,NaN,NaN,NaN,35000,NaN,NaN,Not Fraud
2,6086941026163002,174000,2000,2000,2000,22-MAR-18 12.28.15,0,01-DEC-22 00.00.00,NaN,6011,356,???,544921,944896,000,201,JSEWLK06,1005044921,LXFC BTNK LTD.LUCKNOWUP IN,NaN,NaN,NaN,356,356,356,210201210141,IN1,ATM,0796,NaN,NaN,NaN,44000,NaN,NaN,Not Fraud
3,6086941026163002,174000,1000,1000,1000,22-MAR-18 14.51.45,0,01-DEC-22 00.00.00,NaN,6011,356,???,544921,153505,000,201,JSEWLK06,1005044921,LXFC BTNK LTD.LUCKNOWUP IN,NaN,NaN,NaN,356,356,356,210201210141,IN1,ATM,0796,NaN,NaN,NaN,42000,NaN,NaN,Not Fraud
4,6086941055683003,174000,4500,28.59,2034.06,22-MAR-18 15.50.10,56874000,01-JUL-23 00.00.00,NaN,6011,144,???,544921,757011,000,201,A9021311,1144000031,STMPTTH BTNK TTMCTSHLK,NaN,NaN,NaN,144,840,356,210201210141,IN1,ATM,0797,NaN,NaN,NaN,14791.4,NaN,NaN,Not Fraud
5,6086941020177008,004000,499,7.14,499,22-MAR-18 02.35.44,0,01-DEC-22 00.00.00,NaN,5967,840,???,370100,954345,000,201,FN90G440,9590108460,GOOGLE SERDICES \\MOUNTTIN DIEW\94043 \,NaN,NaN,NaN,356,840,356,200111100040,IN1,POS,0797,NaN,NaN,NaN,177495,NaN,NaN,Not Fraud


In [7]:
data2.shape

(996861, 36)

##### **Removing columns full of NaNs**

In [0]:
indices = []
for i in data2.columns:
  if(pd.isna(data2[i]).sum() == len(data2)):
    indices.append(i)
data2 = data2.drop(indices,axis = 1)

In [9]:
indices

[]

In [10]:
data2.shape

(996861, 36)

##### So we see that none of the columns are full of NaNs, so we retain all 36 of the columns

##### **Dealing with the date type columns**

##### First we will pick out the date type columns. We make this assumption that the columns which are of date type have the term 'DATE' or some form of the word in the name.


In [11]:
names = []
for i in data2.columns:
  names.append(i.upper())
names


['CARD-WALLET-TOKEN ID',
 'PROCESSING_CODE',
 'TRANSACTION_AMOUNT',
 'SETTLEMENT_AMOUNT',
 'BILLING_AMOUNT',
 'TRANSACTION DATE AND TIME',
 'CONVERSION_RATE',
 'EXPIRY DATE',
 'CONVERSION_RATE_DATE',
 'MERCHANT CATEGORY CODE',
 'ACQUIRING_COUNTRY_CODE',
 'POS ENTRY MODE',
 'ACQUIRER_INSTITUTION_CODE',
 'AUTHORIZATION CODE',
 'APPROVAL DECISION',
 'CARD SERVICE CODE',
 'TID',
 'MID',
 'MERCHANT NAME AND ADDRESS',
 'PRIVATE DATA 48',
 'TRANSACTION CATEGORY CODE',
 'DE48 DATA LENGTH',
 'TRANSACTION_CURRENCY',
 'SETTLEMENT_CURRENCY',
 'BILLING_CURRENCY',
 'POS_DATA',
 'INTERFACE ID',
 'TERMINAL TYPE',
 'TOKEN DATA-C1',
 'ADDITIONAL PAYMENT INFO',
 'ECOMMERCE DATA',
 'ECI INDICATOR',
 'REMAINING BALANCE',
 'PRIVATE DATA - ADDITIONAL INFO',
 'TOKEN DATA',
 'TARGET']

In [0]:
data2.columns = names

In [13]:
date_columns = []
for i in names:
  if(i.find('DATE') >= 0):
    date_columns.append(i)
date_columns

['TRANSACTION DATE AND TIME', 'EXPIRY DATE', 'CONVERSION_RATE_DATE']

##### Therefore these are my date columns, we will out checks on them to rule out cases of Fraud.
##### We will compare the transaction date with the current date range and compare it with the conversion rate date which should also be in the vicinity. As for the expiry date we assume that it cannot be before the transaction date and not more than 10 years ahead of the transaction date. Later on we can impose more conditions of similar nature if we want. For this changes will have to be made to the code.

In [14]:
data2['TRANSACTION DATE AND TIME'][1].split(' ')

['22-MAR-18', '09.10.37']

In [15]:
today = date.today()
current_date = today.strftime("%d-%m-%y")
print(current_date)

23-01-20


In [0]:
## creating a dictionary to match the months to their respective serial numbers
month_dict = {'JAN':1,'FEB':2,'MAR':3,'APR':4,'MAY':5,'JUN':6,'JUL':7,'AUG':8,'SEP':9,'OCT':10,'NOV':11,'DEC':12}
## checking is current date is after the transaction date or not because we certainly cannot have a date from the future here
date_1, time_1 = data2['TRANSACTION DATE AND TIME'][1].split(' ')


In [0]:
##splitting the date into month day and year and checking for its validity
def check_transact_date(d):
  today = date.today()
  day, month, year = d.split('-')
  if (int(year)>int(today.strftime("%y"))):
    return 1
  elif(int(year)==int(today.strftime("%y"))) and (month_dict[month]>int(today.strftime("%m"))):
    return 1
  elif(int(year)==int(today.strftime("%y"))) and (month_dict[month]==int(today.strftime("%m"))) and (int(day)>int(today.strftime("%d"))):
    return 1
  else:
    return 0

In [0]:
##checking transaction date and time because it definitely cannot be more than the current time
def check_transact_date_and_time(d,t):
  hours, mins, secs = t.split('.')
  today = date.today()
  day, month, year = d.split('-')
  if(check_transact_date(d)==1): ## checking to see if the date is valid, if not it makes no sense to check time
    return 1
  elif(int(year)==int(today.strftime("%y"))) and (month_dict[month]==int(today.strftime("%m"))) and (int(day)==int(today.strftime("%d"))):
    now = datetime.now()
    if(int(now.strftime("%H"))<int(hours)):
      return 1
    elif(int(now.strftime("%H"))==int(hours)) and (int(now.strftime("%M"))<int(mins)):
      return 1
    elif(int(now.strftime("%H"))==int(hours)) and (int(now.strftime("%M"))==int(mins)) and (int(now.strftime("%S"))<int(secs)):
      return 1
    else:
      return 0
  else:
    return 0

In [19]:
## Now we will find the transactions that are found as frauds on the basis of the above check on transaction date and time
transact_date_fraud_indices = []
data2_temp = data2[pd.isna(data2['TRANSACTION DATE AND TIME'])==False]
data2_temp['new_index'] =  np.arange(1,len(data2_temp)+1,1)
#data2_temp.set_index(index,inplace=True)
data2_temp.set_index(['new_index'], inplace = True) 
start = time.time()
for i in np.arange(1,len(data2_temp),1):
  date_1, time_1 = data2_temp['TRANSACTION DATE AND TIME'][i].split(' ')
  if (check_transact_date_and_time(date_1,time_1)==1):
    transact_date_fraud_indices.append(i)
end = time.time()
print('Time taken to find fraud cases: {}'.format(end-start))
print(len(transact_date_fraud_indices))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Time taken to find fraud cases: 1.2209937572479248
0


##### So we can therefore conclude that there are no frauds in terms of the transaction date

##### As for the conversion rate date we can similarly just apply the 'check_transact_date' function to the find the set of valid dates since the rule applied is the same that it cannot be further ahead than the current date. As for how far it can go in the backward direction, it can be decided upon and changed later.

In [20]:
## Now we will find the frauds that were on the basis of the check on conversion dates
conversion_date_fraud_indices = []
data2_temp = data2[pd.isna(data2['CONVERSION_RATE_DATE'])==False]
data2_temp['new_index'] =  np.arange(1,len(data2_temp)+1,1)
#data2_temp.set_index(index,inplace=True)
data2_temp.set_index(['new_index'], inplace = True) 
start = time.time()
for i in np.arange(1,len(data2_temp),1):
  date_1, time_1 = data2_temp['CONVERSION_RATE_DATE'][i].split(' ')
  if (check_transact_date_and_time(date_1, time_1)==1):
    conversion_date_fraud_indices.append(i)
end = time.time()
print('Time taken to find fraud cases: {}'.format(end-start))
print(len(conversion_date_fraud_indices))

Time taken to find fraud cases: 0.03341245651245117
0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


##### Therefore there are no frauds on the basis of coversion rate dates either

##### Now we will check for frauds on the basis of the expiry date of the transaction method ususally cards. We note that the expiry date cannot be less than the date of the transaction or even more than 10 years ahead, a slight change in function that checks the transaction dates helps us check this. The time is always 00.00.00 so we no longer have to check the time.

In [0]:
def check_expiry_date(d1, d2): ## Here d1 is the expiry date and d2 is the transaction date
  day1, month1, year1 = d1.split('-')
  day2, month2, year2 = d2.split('-')
  if (int(year1)<int(year2)) or (int(year1)>int(year2)+15):
    return 1
  elif(int(year1)==int(year2)) and (month_dict[month1]<month_dict[month2]):
    return 1
  elif(int(year1)==int(year2)) and (month_dict[month1]==month_dict[month2]) and (int(day1)<int(day2)):
    return 1
  else:
    return 0

In [22]:
check_expiry_date('23-SEP-49','25-DEC-20')

1

##### Here the error is that the expiry date is more than 15 years ahead of the transaction date which is usually not the case in real life.

In [23]:
## Now we will find the frauds that were on the basis of expiry dates
expiry_date_fraud_indices = []
data2_temp = data2[pd.isna(data2['EXPIRY DATE'])==False]
data2_temp = data2_temp[pd.isna(data2_temp['TRANSACTION DATE AND TIME'])==False]
data2_temp['new_index'] =  np.arange(1,len(data2_temp)+1,1)
#data2_temp.set_index(index,inplace=True)
data2_temp.set_index(['new_index'], inplace = True) 
start = time.time()
for i in np.arange(1,len(data2_temp),1):
  date_1, time_1 = data2_temp['EXPIRY DATE'][i].split(' ')
  date_2, time_2 = data2_temp['TRANSACTION DATE AND TIME'][i].split(' ')
  if (check_expiry_date(date_1,date_2)==1):
    expiry_date_fraud_indices.append(i)
end = time.time()
print('Time taken to find fraud cases: {}'.format(end-start))
print(len(expiry_date_fraud_indices))

Time taken to find fraud cases: 1.0801365375518799
0


##### Unfortunately there are no data elements which are frauds on the basis of expiry dates as well. We will have to delve further to identify them as frauds. Further on we will be dealing with supervised learning algorithms to identify frauds. So it will be something that looks at fraud data and in the future identifies patterns on the basis of that.

#### **ADABOOST ALGORITHM**

##### Now we will try to use adaptive boosting technique to try and better the results of the Nano-models.
##### First we will start with a small introduction to **AdaBoost**, what it does as an alogirthm, what are the major steps and how it applies to our case here.

>**Step 1:** Initialize the sample weights
In first step of AdaBoost each sample is associated with a weight that indicates how important it is with regards to the classification. Initially, all the samples have identical weights (1 divided by the total number of samples).

>**Step 2:** Build a decision tree with each feature, classify the data and evaluate the result
Next, for each feature, we build a decision tree with a depth of desired length(usually taken as 1 or 2). Then, we use every decision tree to classify the data. Afterwards, we compare the predictions made by each tree with the actual labels in the training set. The feature and corresponding tree that did the best job of classifying the training samples becomes the next tree in the forest.

>**Step 3:** Calculate the significance of the tree in the final classification.
Once we have decided on a decision tree. We use the proceeding formula to calculate the amount of say the it has in the final classification.

$$Significance = \frac{1}{2}*log(\frac{1-total\ error}{toral\ error})$$
>Where the total error is the sum of the weights of the incorrectly classified samples.

> **Step 4:** Update the sample weights so that the next decision tree will take the errors made by the preceding decision tree into account.
We look at the samples that the current tree classified incorrectly and increase their associated weights using the following formula.

$$New\ Sample\ Weight = sample\ weight * e^{significance}$$

>**Step 5:** Form a new dataset. We start by making a new and empty dataset that is the same size as the original. Then, imagine a roulette table where each pocket corresponds to a sample weight. We select numbers between 0 and 1 at random. The location where each number falls determines which sample we place in the new dataset. Since the samples that were incorrectly classified have higher weights in relation to the others, the likelihood that the random number falls under their slice of the distribution is greater. Therefore, the new dataset will have a tendency to contain multiple copies of the samples that were misclassified by the previous tree. As a result, when we go back to the step where we evaluate the predictions made by each decision tree, the one with the highest score will have correctly classified the samples the previous tree misclassified.

>**Step 6:** Repeat steps 2 through 5 until the number of iterations equals the number specified by the hyperparameter (i.e. number of estimators)

>**Step 7:** Use the forest of decision trees to make predictions on data outside of the training set. The AdaBoost model makes predictions by having each tree in the forest classify the sample. Then, we split the trees into groups according to their decisions. For each group, we add up the significance of every tree inside the group. The final classification made by the forest as a whole is determined by the group with the largest sum.



##### Now we will try to use Adaptive Boosting techinque to achieve the classification of transaction as frauds and not frauds.
##### But before that it is important to weed out columns which are not at all helpful such as the date type columns and those which are unique for each row.

##### **Dropping Date columns**

In [24]:
data2_copy = data2.copy()
data2_copy = data2_copy.drop(date_columns,axis = 1)
data2_copy.columns

Index(['CARD-WALLET-TOKEN ID', 'PROCESSING_CODE', 'TRANSACTION_AMOUNT',
       'SETTLEMENT_AMOUNT', 'BILLING_AMOUNT', 'CONVERSION_RATE',
       'MERCHANT CATEGORY CODE', 'ACQUIRING_COUNTRY_CODE', 'POS ENTRY MODE',
       'ACQUIRER_INSTITUTION_CODE', 'AUTHORIZATION CODE', 'APPROVAL DECISION',
       'CARD SERVICE CODE', 'TID', 'MID', 'MERCHANT NAME AND ADDRESS',
       'PRIVATE DATA 48', 'TRANSACTION CATEGORY CODE', 'DE48 DATA LENGTH',
       'TRANSACTION_CURRENCY', 'SETTLEMENT_CURRENCY', 'BILLING_CURRENCY',
       'POS_DATA', 'INTERFACE ID', 'TERMINAL TYPE', 'TOKEN DATA-C1',
       'ADDITIONAL PAYMENT INFO', 'ECOMMERCE DATA', 'ECI INDICATOR',
       'REMAINING BALANCE', 'PRIVATE DATA - ADDITIONAL INFO', 'TOKEN DATA',
       'TARGET'],
      dtype='object')

##### **Removing constant columns and those which are unique to each row**

In [25]:
removable_indices1 = []
removable_indices2 = []
for i in data2_copy.columns:
  if(len(set(data2_copy[i]))==len(data2_copy)):
    removables_indices.append(i)
  elif(len(set(data2_copy))==1):
    removable_indices2.append(i)
data2_copy = data2_copy.drop(removable_indices1,axis=1)
data2_copy = data2_copy.drop(removable_indices2,axis=1)
len(data2_copy.columns)

33

##### So out of 36 columns we are left with 33 columns which we can now input to our AdaBoost classifier. 

In [26]:
data2_copy.dtypes

CARD-WALLET-TOKEN ID              object
PROCESSING_CODE                   object
TRANSACTION_AMOUNT                object
SETTLEMENT_AMOUNT                 object
BILLING_AMOUNT                    object
CONVERSION_RATE                   object
MERCHANT CATEGORY CODE            object
ACQUIRING_COUNTRY_CODE            object
POS ENTRY MODE                    object
ACQUIRER_INSTITUTION_CODE         object
AUTHORIZATION CODE                object
APPROVAL DECISION                 object
CARD SERVICE CODE                 object
TID                               object
MID                               object
MERCHANT NAME AND ADDRESS         object
PRIVATE DATA 48                   object
TRANSACTION CATEGORY CODE         object
DE48 DATA LENGTH                  object
TRANSACTION_CURRENCY              object
SETTLEMENT_CURRENCY               object
BILLING_CURRENCY                  object
POS_DATA                          object
INTERFACE ID                      object
TERMINAL TYPE   

##### Changing question marks in the columns to numeric constants

In [0]:
## for i in data2_copy.columns:
cols = data2_copy.columns
for col in cols:
  l = data2_copy[col]
  new_list = []
  for i in l:
    if pd.isna(i):
      new_list.append(i)
    elif str(i)[0] == '?':
      new_list.append('0')
    else:
      new_list.append(i)
  data2_copy[col] = new_list
# l=data2_copy[cols[8]]
# type(l)
# l.str.count('?')

In [28]:
data2_copy.head()

,CARD-WALLET-TOKEN ID,PROCESSING_CODE,TRANSACTION_AMOUNT,SETTLEMENT_AMOUNT,BILLING_AMOUNT,CONVERSION_RATE,MERCHANT CATEGORY CODE,ACQUIRING_COUNTRY_CODE,POS ENTRY MODE,ACQUIRER_INSTITUTION_CODE,AUTHORIZATION CODE,APPROVAL DECISION,CARD SERVICE CODE,TID,MID,MERCHANT NAME AND ADDRESS,PRIVATE DATA 48,TRANSACTION CATEGORY CODE,DE48 DATA LENGTH,TRANSACTION_CURRENCY,SETTLEMENT_CURRENCY,BILLING_CURRENCY,POS_DATA,INTERFACE ID,TERMINAL TYPE,TOKEN DATA-C1,ADDITIONAL PAYMENT INFO,ECOMMERCE DATA,ECI INDICATOR,REMAINING BALANCE,PRIVATE DATA - ADDITIONAL INFO,TOKEN DATA,TARGET
1,6.086942e+15,174000,7000.0,7000.00,7000.00,0.0,6011.0,356,0,544921,533038,000,201,GYACBB04,1005044921,LXFC BTNK LTD.KHURDT OR IN,NaN,NaN,NaN,356,356,356,210201210141,IN1,ATM,0917,NaN,NaN,NaN,35000.00,NaN,NaN,Not Fraud
2,6.086941e+15,174000,2000.0,2000.00,2000.00,0.0,6011.0,356,0,544921,944896,000,201,JSEWLK06,1005044921,LXFC BTNK LTD.LUCKNOWUP IN,NaN,NaN,NaN,356,356,356,210201210141,IN1,ATM,0796,NaN,NaN,NaN,44000.00,NaN,NaN,Not Fraud
3,6.086941e+15,174000,1000.0,1000.00,1000.00,0.0,6011.0,356,0,544921,153505,000,201,JSEWLK06,1005044921,LXFC BTNK LTD.LUCKNOWUP IN,NaN,NaN,NaN,356,356,356,210201210141,IN1,ATM,0796,NaN,NaN,NaN,42000.00,NaN,NaN,Not Fraud
4,6.086941e+15,174000,4500.0,28.59,2034.06,56874000.0,6011.0,144,0,544921,757011,000,201,A9021311,1144000031,STMPTTH BTNK TTMCTSHLK,NaN,NaN,NaN,144,840,356,210201210141,IN1,ATM,0797,NaN,NaN,NaN,14791.42,NaN,NaN,Not Fraud
5,6.086941e+15,004000,499.0,7.14,499.00,0.0,5967.0,840,0,370100,954345,000,201,FN90G440,9590108460,GOOGLE SERDICES \\MOUNTTIN DIEW\94043 \,NaN,NaN,NaN,356,840,356,200111100040,IN1,POS,0797,NaN,NaN,NaN,177495.26,NaN,NaN,Not Fraud


##### **CHANGING ALPHA-NUMERIC COLUMNS TO NUMERICAL ENCODINGS**

##### **TRIAL**

In [0]:
# pd.to_Categorical(data2_copy['TID'])
l = data2_copy['TID']
l=l[pd.isna(l)==False]
new_col = pd.Categorical(l, categories=l.unique()).codes
df_temp = pd.DataFrame()
df_temp['TID'] = l
df_temp['TID encoded'] = new_col
#df_temp.drop_duplicates()
# # cols = ['TID']
# # data2_copy[cols] = data2_copy[cols].astype('category')


In [0]:
df_temp=df_temp.drop_duplicates()

In [0]:
#data2_copy

In [32]:
df_t=pd.merge(data2_copy, df_temp, how='left', on='TID')
df_t = df_t.drop('TID',axis=1)
df_t = df_t.rename(columns={"TID encoded": "TID"})
df_t

,CARD-WALLET-TOKEN ID,PROCESSING_CODE,TRANSACTION_AMOUNT,SETTLEMENT_AMOUNT,BILLING_AMOUNT,CONVERSION_RATE,MERCHANT CATEGORY CODE,ACQUIRING_COUNTRY_CODE,POS ENTRY MODE,ACQUIRER_INSTITUTION_CODE,AUTHORIZATION CODE,APPROVAL DECISION,CARD SERVICE CODE,MID,MERCHANT NAME AND ADDRESS,PRIVATE DATA 48,TRANSACTION CATEGORY CODE,DE48 DATA LENGTH,TRANSACTION_CURRENCY,SETTLEMENT_CURRENCY,BILLING_CURRENCY,POS_DATA,INTERFACE ID,TERMINAL TYPE,TOKEN DATA-C1,ADDITIONAL PAYMENT INFO,ECOMMERCE DATA,ECI INDICATOR,REMAINING BALANCE,PRIVATE DATA - ADDITIONAL INFO,TOKEN DATA,TARGET,TID
0,6.086942e+15,174000,7000.0,7000.00,7000.00,0.0,6011.0,356,0,544921,533038,000,201,1005044921,LXFC BTNK LTD.KHURDT OR IN,NaN,NaN,NaN,356,356,356,210201210141,IN1,ATM,0917,NaN,NaN,NaN,35000.00,NaN,NaN,Not Fraud,0.0
1,6.086941e+15,174000,2000.0,2000.00,2000.00,0.0,6011.0,356,0,544921,944896,000,201,1005044921,LXFC BTNK LTD.LUCKNOWUP IN,NaN,NaN,NaN,356,356,356,210201210141,IN1,ATM,0796,NaN,NaN,NaN,44000.00,NaN,NaN,Not Fraud,1.0
2,6.086941e+15,174000,1000.0,1000.00,1000.00,0.0,6011.0,356,0,544921,153505,000,201,1005044921,LXFC BTNK LTD.LUCKNOWUP IN,NaN,NaN,NaN,356,356,356,210201210141,IN1,ATM,0796,NaN,NaN,NaN,42000.00,NaN,NaN,Not Fraud,1.0
3,6.086941e+15,174000,4500.0,28.59,2034.06,56874000.0,6011.0,144,0,544921,757011,000,201,1144000031,STMPTTH BTNK TTMCTSHLK,NaN,NaN,NaN,144,840,356,210201210141,IN1,ATM,0797,NaN,NaN,NaN,14791.42,NaN,NaN,Not Fraud,2.0
4,6.086941e+15,004000,499.0,7.14,499.00,0.0,5967.0,840,0,370100,954345,000,201,9590108460,GOOGLE SERDICES \\MOUNTTIN DIEW\94043 \,NaN,NaN,NaN,356,840,356,200111100040,IN1,POS,0797,NaN,NaN,NaN,177495.26,NaN,NaN,Not Fraud,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996856,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Fraud,NaN
996857,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Fraud,NaN
996858,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Fraud,NaN
996859,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Fraud,NaN


##### **Doing this for all columns now**

In [33]:
start = time.time()
for col in data2_copy.columns:
  l = data2_copy[col]
  l=l[pd.isna(l)==False]
  new_col = pd.Categorical(l, categories=l.unique()).codes
  df_temp = pd.DataFrame()
  df_temp[col] = l
  df_temp[col+' encoded'] = new_col
  df_temp=df_temp.drop_duplicates()
  data2_copy=pd.merge(data2_copy, df_temp, how='left', on=col)
  data2_copy = data2_copy.drop(col,axis=1)
  data2_copy = data2_copy.rename(columns={col+' encoded': col})
end = time.time()
print("The time taken to encode all the columns are:{}".format(end-start))

The time taken to encode all the columns are:51.54670596122742


In [34]:
data2_copy

,CARD-WALLET-TOKEN ID,PROCESSING_CODE,TRANSACTION_AMOUNT,SETTLEMENT_AMOUNT,BILLING_AMOUNT,CONVERSION_RATE,MERCHANT CATEGORY CODE,ACQUIRING_COUNTRY_CODE,POS ENTRY MODE,ACQUIRER_INSTITUTION_CODE,AUTHORIZATION CODE,APPROVAL DECISION,CARD SERVICE CODE,TID,MID,MERCHANT NAME AND ADDRESS,PRIVATE DATA 48,TRANSACTION CATEGORY CODE,DE48 DATA LENGTH,TRANSACTION_CURRENCY,SETTLEMENT_CURRENCY,BILLING_CURRENCY,POS_DATA,INTERFACE ID,TERMINAL TYPE,TOKEN DATA-C1,ADDITIONAL PAYMENT INFO,ECOMMERCE DATA,ECI INDICATOR,REMAINING BALANCE,PRIVATE DATA - ADDITIONAL INFO,TOKEN DATA,TARGET
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,0
1,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,0
2,1.0,0.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,1.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,2.0,NaN,NaN,0
3,2.0,0.0,3.0,3.0,3.0,1.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0,2.0,1.0,2.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,0.0,2.0,NaN,NaN,NaN,3.0,NaN,NaN,0
4,3.0,1.0,4.0,4.0,4.0,0.0,1.0,2.0,0.0,1.0,4.0,0.0,0.0,3.0,2.0,3.0,NaN,NaN,NaN,0.0,1.0,0.0,1.0,0.0,1.0,2.0,NaN,NaN,NaN,4.0,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996856,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
996857,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
996858,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
996859,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


##### Since removing all the rows with NaNs leaves us with no rows at all, we will choose to fill with the modal(highest occuring) value in the column. This might affect accuracy but will not be a problem when the data is free of NaNs.

In [45]:
##creating mode value dictionary
mode_dict = {}
for col in data2_copy.columns:
  mode_dict[col] = data2_copy[col].mode()[0]
mode_dict

{'ACQUIRER_INSTITUTION_CODE': 20.0,
 'ACQUIRING_COUNTRY_CODE': 0.0,
 'ADDITIONAL PAYMENT INFO': 0.0,
 'APPROVAL DECISION': 0.0,
 'AUTHORIZATION CODE': 554.0,
 'BILLING_AMOUNT': 2.0,
 'BILLING_CURRENCY': 0.0,
 'CARD SERVICE CODE': 1.0,
 'CARD-WALLET-TOKEN ID': 4476.0,
 'CONVERSION_RATE': 3.0,
 'DE48 DATA LENGTH': 2.0,
 'ECI INDICATOR': 0.0,
 'ECOMMERCE DATA': 0.0,
 'INTERFACE ID': 1.0,
 'MERCHANT CATEGORY CODE': 12.0,
 'MERCHANT NAME AND ADDRESS': 884.0,
 'MID': 579.0,
 'POS ENTRY MODE': 0.0,
 'POS_DATA': 29.0,
 'PRIVATE DATA - ADDITIONAL INFO': 255.0,
 'PRIVATE DATA 48': 11.0,
 'PROCESSING_CODE': 4.0,
 'REMAINING BALANCE': 537.0,
 'SETTLEMENT_AMOUNT': 97.0,
 'SETTLEMENT_CURRENCY': 0.0,
 'TARGET': 0,
 'TERMINAL TYPE': 1.0,
 'TID': 156.0,
 'TOKEN DATA': 0.0,
 'TOKEN DATA-C1': 1.0,
 'TRANSACTION CATEGORY CODE': 1.0,
 'TRANSACTION_AMOUNT': 93.0,
 'TRANSACTION_CURRENCY': 0.0}

In [58]:
for col in data2_copy.columns:
  data2_copy[col] = data2_copy[col].where(pd.isna(data2_copy[col])==False,mode_dict[col])
data2_copy

,CARD-WALLET-TOKEN ID,PROCESSING_CODE,TRANSACTION_AMOUNT,SETTLEMENT_AMOUNT,BILLING_AMOUNT,CONVERSION_RATE,MERCHANT CATEGORY CODE,ACQUIRING_COUNTRY_CODE,POS ENTRY MODE,ACQUIRER_INSTITUTION_CODE,AUTHORIZATION CODE,APPROVAL DECISION,CARD SERVICE CODE,TID,MID,MERCHANT NAME AND ADDRESS,PRIVATE DATA 48,TRANSACTION CATEGORY CODE,DE48 DATA LENGTH,TRANSACTION_CURRENCY,SETTLEMENT_CURRENCY,BILLING_CURRENCY,POS_DATA,INTERFACE ID,TERMINAL TYPE,TOKEN DATA-C1,ADDITIONAL PAYMENT INFO,ECOMMERCE DATA,ECI INDICATOR,REMAINING BALANCE,PRIVATE DATA - ADDITIONAL INFO,TOKEN DATA,TARGET
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,255.0,0.0,0
1,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,11.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,255.0,0.0,0
2,1.0,0.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,1.0,11.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,255.0,0.0,0
3,2.0,0.0,3.0,3.0,3.0,1.0,0.0,1.0,0.0,0.0,3.0,0.0,0.0,2.0,1.0,2.0,11.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,3.0,255.0,0.0,0
4,3.0,1.0,4.0,4.0,4.0,0.0,1.0,2.0,0.0,1.0,4.0,0.0,0.0,3.0,2.0,3.0,11.0,1.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,2.0,0.0,0.0,0.0,4.0,255.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996856,4476.0,4.0,93.0,97.0,2.0,3.0,12.0,0.0,0.0,20.0,554.0,0.0,1.0,156.0,579.0,884.0,11.0,1.0,2.0,0.0,0.0,0.0,29.0,1.0,1.0,1.0,0.0,0.0,0.0,537.0,255.0,0.0,0
996857,4476.0,4.0,93.0,97.0,2.0,3.0,12.0,0.0,0.0,20.0,554.0,0.0,1.0,156.0,579.0,884.0,11.0,1.0,2.0,0.0,0.0,0.0,29.0,1.0,1.0,1.0,0.0,0.0,0.0,537.0,255.0,0.0,0
996858,4476.0,4.0,93.0,97.0,2.0,3.0,12.0,0.0,0.0,20.0,554.0,0.0,1.0,156.0,579.0,884.0,11.0,1.0,2.0,0.0,0.0,0.0,29.0,1.0,1.0,1.0,0.0,0.0,0.0,537.0,255.0,0.0,0
996859,4476.0,4.0,93.0,97.0,2.0,3.0,12.0,0.0,0.0,20.0,554.0,0.0,1.0,156.0,579.0,884.0,11.0,1.0,2.0,0.0,0.0,0.0,29.0,1.0,1.0,1.0,0.0,0.0,0.0,537.0,255.0,0.0,0


In [0]:
# #values = mode_dict
# data2_copy.fillna(value=mode_dict)
# data2_copy

##### The problem with this is that it changes the NaNs to a new column which cannot be right because the NaNs could it reality be a range of values. The AdaBoost classifier can handle NaNs so what we must try and do is change everything other than the NaNs to a new numeric encoding. We might have to do this in linear time although we will try our best to do better than that.

In [0]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_breast_cancer
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder

In [0]:
X = pd.DataFrame(data2_copy.iloc[:,:32], columns=data2_copy.iloc[:,:32].columns)
y = pd.Categorical.from_codes(data2_copy.iloc[:,32], ['Not Fraud', 'Fraud'])

In [62]:
data2_copy.iloc[:,:32].columns

Index(['CARD-WALLET-TOKEN ID', 'PROCESSING_CODE', 'TRANSACTION_AMOUNT',
       'SETTLEMENT_AMOUNT', 'BILLING_AMOUNT', 'CONVERSION_RATE',
       'MERCHANT CATEGORY CODE', 'ACQUIRING_COUNTRY_CODE', 'POS ENTRY MODE',
       'ACQUIRER_INSTITUTION_CODE', 'AUTHORIZATION CODE', 'APPROVAL DECISION',
       'CARD SERVICE CODE', 'TID', 'MID', 'MERCHANT NAME AND ADDRESS',
       'PRIVATE DATA 48', 'TRANSACTION CATEGORY CODE', 'DE48 DATA LENGTH',
       'TRANSACTION_CURRENCY', 'SETTLEMENT_CURRENCY', 'BILLING_CURRENCY',
       'POS_DATA', 'INTERFACE ID', 'TERMINAL TYPE', 'TOKEN DATA-C1',
       'ADDITIONAL PAYMENT INFO', 'ECOMMERCE DATA', 'ECI INDICATOR',
       'REMAINING BALANCE', 'PRIVATE DATA - ADDITIONAL INFO', 'TOKEN DATA'],
      dtype='object')

In [0]:
train_X, test_X, train_y, test_y = train_test_split(X, y, random_state=1)

In [66]:
start = time.time()
classifier = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=2),## This number signifies that atmost 3 variables are considered together
    n_estimators=200
)
classifier.fit(train_X, train_y)
end = time.time()
print('The time taken to fit the classifier is:{}'.format(end-start))

The time taken to fit the classifier is:338.01833868026733


In [0]:
predictions = classifier.predict(test_X)

In [68]:
predictions

array(['Not Fraud', 'Not Fraud', 'Not Fraud', ..., 'Not Fraud',
       'Not Fraud', 'Not Fraud'], dtype=object)

In [69]:
## This is the mean accuracy on the test data
classifier.score(test_X,test_y)

0.9999959874165383

In [70]:
classifier.feature_importances_

array([0.06394762, 0.00017962, 0.040494  , 0.07513438, 0.05387596,
       0.00099546, 0.06229796, 0.01369416, 0.00256526, 0.02069924,
       0.00172725, 0.07169493, 0.01129204, 0.0464718 , 0.07185426,
       0.0563174 , 0.05007575, 0.01409484, 0.03159223, 0.00400835,
       0.01153899, 0.01427408, 0.03609613, 0.        , 0.00379384,
       0.07703228, 0.03855063, 0.        , 0.        , 0.04410874,
       0.08159279, 0.        ])

In [73]:
sum(classifier.feature_importances_)

0.9999999999999999

##### Now depending on the number of features we want to select finally(say k), we have pick the k largest values from the above list and find the predictors corresponding to it.

In [76]:
## The most important column
data2_copy.columns[np.argmax(classifier.feature_importances_)]

'PRIVATE DATA - ADDITIONAL INFO'

In [81]:
## The k most important columns, here k =5
importances = classifier.feature_importances_
k = 5
idx = np.argpartition(importances, -k)
importances[idx[-k:]]


array([0.07169493, 0.07185426, 0.07513438, 0.08159279, 0.07703228])

In [85]:
l = []
for i in importances[idx[-k:]]:
  l.append(data2_copy.columns[np.where(importances==i)])
l

[Index(['APPROVAL DECISION'], dtype='object'),
 Index(['MID'], dtype='object'),
 Index(['SETTLEMENT_AMOUNT'], dtype='object'),
 Index(['PRIVATE DATA - ADDITIONAL INFO'], dtype='object'),
 Index(['TOKEN DATA-C1'], dtype='object')]

##### This gives us the most significant columns in the decision making process, we shoudl however take this with a pinch of salt since we have filled in quite a number of values. On a filled dataset this would be much much more reliable as an output.

##### **ROUGH WORK**

In [0]:
df_temp = pd.DataFrame(data2_copy['TID'])
#encoder = LabelEncoder()
binary_encoded_y = pd.Categorical(df_temp['TID'], categories=df_temp['TID'].unique()).codes
df_temp['TID_encoded'] = binary_encoded_y
df_temp

ValueError: ignored

In [0]:
df_temp

,TID,TID_encoded
1,GYACBB04,25033.0
2,JSEWLK06,25033.0
3,JSEWLK06,24372.0
4,A9021311,24819.0
5,FN90G440,25505.0
...,...,...
996857,NaN,26293.0
996858,NaN,26293.0
996859,NaN,26293.0
996860,NaN,26293.0


In [0]:
#binary_encoded_y = pd.Series(encoder.fit_transform(data2_copy['TID'].astype('str')))

In [0]:
#binary_encoded_y

0         24852
1         25033
2         25033
3         24372
4         24819
          ...  
996856    26293
996857    26293
996858    26293
996859    26293
996860    26293
Length: 996861, dtype: int64

In [0]:
#new_col

In [0]:
data2_copy.apply(pd.to_numeric)
data2_copy.head()

ValueError: ignored

In [0]:
data2_copy.head()

,CARD-WALLET-TOKEN ID,PROCESSING_CODE,TRANSACTION_AMOUNT,SETTLEMENT_AMOUNT,BILLING_AMOUNT,CONVERSION_RATE,MERCHANT CATEGORY CODE,ACQUIRING_COUNTRY_CODE,POS ENTRY MODE,ACQUIRER_INSTITUTION_CODE,AUTHORIZATION CODE,APPROVAL DECISION,CARD SERVICE CODE,TID,MID,MERCHANT NAME AND ADDRESS,PRIVATE DATA 48,TRANSACTION CATEGORY CODE,DE48 DATA LENGTH,TRANSACTION_CURRENCY,SETTLEMENT_CURRENCY,BILLING_CURRENCY,POS_DATA,INTERFACE ID,TERMINAL TYPE,TOKEN DATA-C1,ADDITIONAL PAYMENT INFO,ECOMMERCE DATA,ECI INDICATOR,REMAINING BALANCE,PRIVATE DATA - ADDITIONAL INFO,TOKEN DATA,TARGET
1,6086941819072006,174000,7000,7000,7000,0,6011,356,???,544921,533038,000,201,GYACBB04,1005044921,LXFC BTNK LTD.KHURDT OR IN,NaN,NaN,NaN,356,356,356,210201210141,IN1,ATM,0917,NaN,NaN,NaN,35000,NaN,NaN,Not Fraud
2,6086941026163002,174000,2000,2000,2000,0,6011,356,???,544921,944896,000,201,JSEWLK06,1005044921,LXFC BTNK LTD.LUCKNOWUP IN,NaN,NaN,NaN,356,356,356,210201210141,IN1,ATM,0796,NaN,NaN,NaN,44000,NaN,NaN,Not Fraud
3,6086941026163002,174000,1000,1000,1000,0,6011,356,???,544921,153505,000,201,JSEWLK06,1005044921,LXFC BTNK LTD.LUCKNOWUP IN,NaN,NaN,NaN,356,356,356,210201210141,IN1,ATM,0796,NaN,NaN,NaN,42000,NaN,NaN,Not Fraud
4,6086941055683003,174000,4500,28.59,2034.06,56874000,6011,144,???,544921,757011,000,201,A9021311,1144000031,STMPTTH BTNK TTMCTSHLK,NaN,NaN,NaN,144,840,356,210201210141,IN1,ATM,0797,NaN,NaN,NaN,14791.4,NaN,NaN,Not Fraud
5,6086941020177008,004000,499,7.14,499,0,5967,840,???,370100,954345,000,201,FN90G440,9590108460,GOOGLE SERDICES \\MOUNTTIN DIEW\94043 \,NaN,NaN,NaN,356,840,356,200111100040,IN1,POS,0797,NaN,NaN,NaN,177495,NaN,NaN,Not Fraud


In [0]:
set(data2_copy['POS ENTRY MODE'])

{'050', '051', '071', '801', '900', '901', '902', '???', nan}

In [0]:
data2_copy['POS ENTRY MODE'] = data2_copy['POS ENTRY MODE'].replace('???','000')

In [0]:
data2_copy['AUTHORIZATION CODE'] = data2_copy['AUTHORIZATION CODE'].replace('??????','000000')

In [0]:
data2_copy.dtypes

CARD-WALLET-TOKEN ID              object
PROCESSING_CODE                   object
TRANSACTION_AMOUNT                object
SETTLEMENT_AMOUNT                 object
BILLING_AMOUNT                    object
CONVERSION_RATE                   object
MERCHANT CATEGORY CODE            object
ACQUIRING_COUNTRY_CODE            object
POS ENTRY MODE                    object
ACQUIRER_INSTITUTION_CODE         object
AUTHORIZATION CODE                object
APPROVAL DECISION                 object
CARD SERVICE CODE                 object
TID                               object
MID                               object
MERCHANT NAME AND ADDRESS         object
PRIVATE DATA 48                   object
TRANSACTION CATEGORY CODE         object
DE48 DATA LENGTH                  object
TRANSACTION_CURRENCY              object
SETTLEMENT_CURRENCY               object
BILLING_CURRENCY                  object
POS_DATA                          object
INTERFACE ID                      object
TERMINAL TYPE   

In [0]:
'bc'.find('a')

-1

In [0]:
data2.reset_index(inplace=True,drop=True)
data2_temp.head()

,CARD-WALLET-TOKEN ID,PROCESSING_CODE,TRANSACTION_AMOUNT,SETTLEMENT_AMOUNT,BILLING_AMOUNT,TRANSACTION DATE AND TIME,CONVERSION_RATE,EXPIRY DATE,CONVERSION_RATE_DATE,MERCHANT CATEGORY CODE,ACQUIRING_COUNTRY_CODE,POS ENTRY MODE,ACQUIRER_INSTITUTION_CODE,AUTHORIZATION CODE,APPROVAL DECISION,CARD SERVICE CODE,TID,MID,MERCHANT NAME AND ADDRESS,PRIVATE DATA 48,TRANSACTION CATEGORY CODE,DE48 DATA LENGTH,TRANSACTION_CURRENCY,SETTLEMENT_CURRENCY,BILLING_CURRENCY,POS_DATA,INTERFACE ID,TERMINAL TYPE,TOKEN DATA-C1,ADDITIONAL PAYMENT INFO,ECOMMERCE DATA,ECI INDICATOR,REMAINING BALANCE,PRIVATE DATA - ADDITIONAL INFO,TOKEN DATA,TARGET
664,6086941023394006,004000,182.03,12518.2,12518.2,22-MAR-18 03.34.13,1,01-DEC-22 00.00.00,22-JUL-18 00.00.00,7011,834,???,370100,??????,117,201,2S90GE04,9734114549,KISIWT ON THE BE301579\\ZTNZIBTR\\\834,NaN,NaN,NaN,840,356,356,510101513301,IN1,POS,0784,NaN,NaN,NaN,NaN,NaN,NaN,Fraud
41769,6854591729122000,000000,3500,50.91,3500,22-MAR-18 01.09.34,0,01-MAY-23 00.00.00,21-JUL-18 00.00.00,7011,356,051,007064,??????,100,206,90106669,THE ONTIME HOTE,THE ONTIME HOTEL KALYAN IND,???,?,??,356,840,356,310101500001,IN9,POS,0153,MPLP2FNP5,NaN,NaN,NaN,0000000000900356400059,NaN,Not Fraud
41784,6854593019894000,000000,49000,712.73,49000,22-MAR-18 10.39.43,0,01-MAR-23 00.00.00,21-JUL-18 00.00.00,5812,356,051,017014,??????,100,226,42263068,42MS00000286253,MSW*DICTORY ENTERPRISE Bangalore IND,????,?,???,356,840,356,310101500001,IN9,POS,NaN,MPL2C5RJC,NaN,NaN,NaN,0000000009500356560028,NaN,Fraud
41785,6854592107326000,171000,5000,72.73,5000,22-MAR-18 10.41.17,0,01-JUN-23 00.00.00,21-JUL-18 00.00.00,6011,356,901,088792,??????,100,206,90328012,SBI BHANDARA AT,SBI BHANDARA ATM 12 THARSA IND,???,?,??,356,840,356,210301200001,IN9,POS,NaN,MPL1UVCLA,NaN,NaN,NaN,1010000001200356441104,NaN,Fraud
41786,6854592107326000,171000,10000,145.45,10000,22-MAR-18 10.41.48,0,01-JUN-23 00.00.00,21-JUL-18 00.00.00,6011,356,901,088792,??????,100,206,90328012,SBI BHANDARA AT,SBI BHANDARA ATM 12 THARSA IND,???,?,??,356,840,356,210301200001,IN9,POS,NaN,MPL1VKDDS,NaN,NaN,NaN,1010000001200356441104,NaN,Fraud


In [0]:
data2_temp['CONVERSION_RATE_DATE'][664]

'22-JUL-18 00.00.00'

In [0]:
pd.isna(data2['CONVERSION_RATE_DATE']).sum()

995661

In [0]:
('abs123').upper()

'ABS123'

In [0]:
'geeks'.find('e')

1

In [0]:
check_transact_date(date_1)

0

In [0]:
check_transact_date_and_time(date_1,time_1)

0

In [0]:
check_transact_date_and_time(d='13-JAN-20',t='22.30.10')

1

In [0]:
func = lambda s: s[:2].upper() + s[2:].lower() if s else ''
date, time = data2['TRANSACTION DATE AND TIME'][1].split(' ')
func(date)

'22-mar-18'

In [0]:


# dd/mm/YY
d1 = today.strftime("%d/%m/%Y")
print("d1 =", d1)
# Textual month, day and year	
d2 = today.strftime("%B %d, %Y")
print("d2 =", d2)
# mm/dd/y
d3 = today.strftime("%m/%d/%y")
print("d3 =", d3)
# Month abbreviation, day and year	
d4 = today.strftime("%b-%d-%y")
print("d4 =", d4)

d1 = 13/01/2020
d2 = January 13, 2020
d3 = 01/13/20
d4 = Jan-13-20


In [0]:
from datetime import datetime
# datetime object containing current date and time
now = datetime.now()
 
print("now =", now)
# dd/mm/YY H:M:S
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)

now = 2020-01-13 14:23:54.203888
date and time = 13/01/2020 14:23:54


In [0]:
data2_temp = data2[pd.isna(data2['TRANSACTION DATE AND TIME'])==False]
data2_temp.head()

,CARD-WALLET-TOKEN ID,PROCESSING_CODE,TRANSACTION_AMOUNT,SETTLEMENT_AMOUNT,BILLING_AMOUNT,TRANSACTION DATE AND TIME,CONVERSION_RATE,EXPIRY DATE,CONVERSION_RATE_DATE,MERCHANT CATEGORY CODE,ACQUIRING_COUNTRY_CODE,POS ENTRY MODE,ACQUIRER_INSTITUTION_CODE,AUTHORIZATION CODE,APPROVAL DECISION,CARD SERVICE CODE,TID,MID,MERCHANT NAME AND ADDRESS,PRIVATE DATA 48,TRANSACTION CATEGORY CODE,DE48 DATA LENGTH,TRANSACTION_CURRENCY,SETTLEMENT_CURRENCY,BILLING_CURRENCY,POS_DATA,INTERFACE ID,TERMINAL TYPE,TOKEN DATA-C1,ADDITIONAL PAYMENT INFO,ECOMMERCE DATA,ECI INDICATOR,REMAINING BALANCE,PRIVATE DATA - ADDITIONAL INFO,TOKEN DATA,TARGET
1,6086941819072006,174000,7000,7000,7000,22-MAR-18 09.10.37,0,01-JUL-23 00.00.00,NaN,6011,356,???,544921,533038,000,201,GYACBB04,1005044921,LXFC BTNK LTD.KHURDT OR IN,NaN,NaN,NaN,356,356,356,210201210141,IN1,ATM,0917,NaN,NaN,NaN,35000,NaN,NaN,Not Fraud
2,6086941026163002,174000,2000,2000,2000,22-MAR-18 12.28.15,0,01-DEC-22 00.00.00,NaN,6011,356,???,544921,944896,000,201,JSEWLK06,1005044921,LXFC BTNK LTD.LUCKNOWUP IN,NaN,NaN,NaN,356,356,356,210201210141,IN1,ATM,0796,NaN,NaN,NaN,44000,NaN,NaN,Not Fraud
3,6086941026163002,174000,1000,1000,1000,22-MAR-18 14.51.45,0,01-DEC-22 00.00.00,NaN,6011,356,???,544921,153505,000,201,JSEWLK06,1005044921,LXFC BTNK LTD.LUCKNOWUP IN,NaN,NaN,NaN,356,356,356,210201210141,IN1,ATM,0796,NaN,NaN,NaN,42000,NaN,NaN,Not Fraud
4,6086941055683003,174000,4500,28.59,2034.06,22-MAR-18 15.50.10,56874000,01-JUL-23 00.00.00,NaN,6011,144,???,544921,757011,000,201,A9021311,1144000031,STMPTTH BTNK TTMCTSHLK,NaN,NaN,NaN,144,840,356,210201210141,IN1,ATM,0797,NaN,NaN,NaN,14791.4,NaN,NaN,Not Fraud
5,6086941020177008,004000,499,7.14,499,22-MAR-18 02.35.44,0,01-DEC-22 00.00.00,NaN,5967,840,???,370100,954345,000,201,FN90G440,9590108460,GOOGLE SERDICES \\MOUNTTIN DIEW\94043 \,NaN,NaN,NaN,356,840,356,200111100040,IN1,POS,0797,NaN,NaN,NaN,177495,NaN,NaN,Not Fraud


In [0]:
breast_cancer = load_breast_cancer()


In [0]:
X = pd.DataFrame(breast_cancer.data, columns=breast_cancer.feature_names)
y = pd.Categorical.from_codes(breast_cancer.target, breast_cancer.target_names)

In [0]:
X.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [0]:
type(y)

pandas.core.arrays.categorical.Categorical

In [0]:
breast_cancer

{'DESCR': '.. _breast_cancer_dataset:\n\nBreast cancer wisconsin (diagnostic) dataset\n--------------------------------------------\n\n**Data Set Characteristics:**\n\n    :Number of Instances: 569\n\n    :Number of Attributes: 30 numeric, predictive attributes and the class\n\n    :Attribute Information:\n        - radius (mean of distances from center to points on the perimeter)\n        - texture (standard deviation of gray-scale values)\n        - perimeter\n        - area\n        - smoothness (local variation in radius lengths)\n        - compactness (perimeter^2 / area - 1.0)\n        - concavity (severity of concave portions of the contour)\n        - concave points (number of concave portions of the contour)\n        - symmetry \n        - fractal dimension ("coastline approximation" - 1)\n\n        The mean, standard error, and "worst" or largest (mean of the three\n        largest values) of these features were computed for each image,\n        resulting in 30 features.  For

In [0]:
encoder = LabelEncoder()
binary_encoded_y = pd.Series(encoder.fit_transform(y))

In [0]:
binary_encoded_y

0      1
1      1
2      1
3      1
4      1
      ..
564    1
565    1
566    1
567    1
568    0
Length: 569, dtype: int64

In [0]:
train_X, test_X, train_y, test_y = train_test_split(X, binary_encoded_y, random_state=1)

In [0]:
classifier = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1),
    n_estimators=200
)
classifier.fit(train_X, train_y)

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                         class_weight=None,
                                                         criterion='gini',
                                                         max_depth=1,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort='deprecated',
                          

In [0]:
predictions = classifier.predict(test_X)

In [0]:
predictions

array([0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0,
       1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0,
       0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0])

In [0]:
classifier.score(test_X,test_y)

0.965034965034965